In [1]:
from pydantic import BaseModel
from openai import OpenAI

In [8]:
class Step(BaseModel):
    explanation: str
    output: str


class MathResponse(BaseModel):
    steps: list[Step]
    final_answer: str


client = OpenAI()

completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a helpful math tutor."},
        {"role": "user", "content": "solve 8x + 31 = 2"},
    ],
    response_format=MathResponse,
)

message = completion.choices[0].message
if message.parsed:
    print(message.parsed.steps)
    print(message.parsed.final_answer)
else:
    print(message.refusal)

[Step(explanation='Subtract 31 from both sides to isolate the term with x.', output='8x = 2 - 31'), Step(explanation='Simplify the right side by performing the subtraction.', output='8x = -29'), Step(explanation='Divide both sides by 8 to solve for x.', output='x = -29/8')]
x = -29/8


In [10]:
class RemoteSynthesisResponse(BaseModel):
    explanation: str
    feedback: str | None
    decision: str
    answer: str | None
    scratchpad: str

In [12]:
REMOTE_SYNTHESIS_JSON = """\
I have enough information to answer the question. The text clearly indicates the names of three researchers: Dan, Filipe, and Pedro.
All are involved in the project. Therefore I will provide the final answer.

Based on your step-by-step analysis above, please strictly return a single object with the following fields:

```python
class RemoteSynthesisResponse(BaseModel):
    explanation: str # A concise statement of your reasoning
    feedback: str | None # If more information is needed, what specifically to look for. NO CODE in this field. Set to null if no feedback needed.
    decision: str # Must be either 'provide_final_answer' OR 'request_additional_info'
    answer: str | None # The final answer if decision is 'provide_final_answer', null otherwise
    scratchpad: str # Brief summary of all information gathered so far, including your current analysis. This will be shown in future rounds.
```
"""

In [16]:
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": REMOTE_SYNTHESIS_JSON},
    ],
    response_format=RemoteSynthesisResponse,
)

In [18]:
message = completion.choices[0].message
if message.parsed:
    print(message.parsed.explanation)
    print(message.parsed.answer)
else:
    print(message.refusal)

All three researchers, Dan, Filipe, and Pedro, are clearly mentioned as participants in the project.
Dan, Filipe, and Pedro are all involved in the project.
